### 1、初始化 Azure API 调用环境


In [22]:
import os
import sys               
sys.path.append(os.path.dirname(os.path.abspath("."))) 
from API_KEYS import InitAzureEnv,AzureDeploymentName

InitAzureEnv()

### 2、获取并存储宽表的表结构

#### 2.1、初始化数据库链接


In [23]:
import pymysql
conn = pymysql.connect(
    host="localhost", 
    port=3306,
    user="root", 
    password="123456", 
    db="demo"
)

#### 2.2、获取所有系统


In [25]:
cursor = conn.cursor()

#### 2.3、获取所有表


In [26]:
cursor.execute('''SELECT 
        ApplicationName 'AppName',
        mdc_app.appcode 'AppCode',
        mdc_table.TableName 'TableCode',
        mdc_table.DisplayName 'TableName'
    FROM mdc_table
    INNER JOIN mdc_app ON mdc_table.appcode = mdc_app.appcode and mdc_table.AppLevelCode = mdc_app.levelcode
    where mdc_app.levelcode like '1000%'
    ORDER BY  mdc_app.levelcode''')
tables = cursor.fetchall()

# list_group=[]

# for app in apps: 
#     list_group.append([app[2],[x[3]+ "("+ x[2] +")" for x in tables if x[1] == app[0]]])

# f = open("table_infos.txt", "a")
# for ag in list_group:
#     if len(ag[1])>0:
#         f.write("`{0}`系统有以下数据表：{1}\n\n".format(ag[0],','.join(ag[1])))
# f.close()

In [27]:
print(tables[:3])

(('销售系统', '0011', 'data_wide_s_Opportunity', '机会宽表'), ('销售系统', '0011', 'data_wide_s_AccountAge', '应收账龄宽表'), ('销售系统', '0011', 'data_wide_dws_s_SalesPerf', '销售业绩汇总宽表'))


#### 2.3、获取所有字段


In [37]:
cursor.execute('''SELECT
        mdc_table.displayName 'tableCnName',
        mdc_table.TableName,
        mdc_tablecolumn.displayName 'ColumnCnName',
        ColumnName
    FROM mdc_table
    INNER JOIN mdc_app ON mdc_table.appcode = mdc_app.appcode and mdc_table.AppLevelCode = mdc_app.levelcode
    INNER JOIN mdc_tablecolumn on mdc_tablecolumn.TableName = mdc_table.TableName
    where mdc_app.AppCode = '0011' and mdc_table.TableName in ('data_wide_s_Room','data_wide_s_Trade')
    ORDER BY  TableName''')
columns = cursor.fetchall()

# col_group=[]

# for tab in tables: 
#     col_group.append([tab[3],[x[1]+ "("+ x[2] +")" for x in columns if x[0] == tab[2]]])

# f = open("table_infos.txt", "a")
# for col in col_group:
#     # print("`{0}`有以下字段：{1}\n\n".format(col[0],','.join(col[1])))
#     if len(col[1])>0:
#         f.write("`{0}`表有以下字段：{1}\n\n".format(col[0],','.join(col[1])))
# f.close()

In [39]:
coljsons = [dict(zip(("tc", "te", "cc","ce"), x)) for x in columns[:5]]
print(coljsons)

[{'tc': '房间宽表', 'te': 'data_wide_s_Room', 'cc': '补充协议总价', 'ce': 'BcxyTotal'}, {'tc': '房间宽表', 'te': 'data_wide_s_Room', 'cc': '实际交房日期', 'ce': 'SjjfDate'}, {'tc': '房间宽表', 'te': 'data_wide_s_Room', 'cc': '标准建筑单价', 'ce': 'BldPrice'}, {'tc': '房间宽表', 'te': 'data_wide_s_Room', 'cc': '楼层', 'ce': 'Floor'}, {'tc': '房间宽表', 'te': 'data_wide_s_Room', 'cc': '项目名称', 'ce': 'ParentProjName'}]


### 3、将 tableinfos 插入都本地数据库


In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import json
from langchain.docstore.document import Document

embeddings = OpenAIEmbeddings(
    deployment=AzureDeploymentName.EMEDDINGS,
    chunk_size=1,
)
chroma_directory = os.path.abspath("chroma/sql")

pages = []
for cj in coljsons:
    pages.append(Document(page_content=json.dumps(cj,ensure_ascii=False),metadata=dict()))

docsearch = Chroma.from_documents(pages, embeddings, persist_directory=chroma_directory)
docsearch.persist()

Using embedded DuckDB with persistence: data will be stored in: /Users/jiangyy/Codes/AI/LangChainCn/03-Samples/chroma/sql


### 4、生成查询 SQL


In [56]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate

# # 加载数据
docsearch = Chroma(persist_directory=chroma_directory, embedding_function=embeddings)


query = '''这段json中tc表示数据表中文名, te标书数据表名, cc表示字段中文名, ce表示字段名。
交易宽表的表名是什么？ '''

docss = docsearch.similarity_search(query)

llm = AzureOpenAI(
    deployment_name=AzureDeploymentName.LLM,
    temperature=0
)

chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
chain.run(input_documents=docss, question=query)

Using embedded DuckDB with persistence: data will be stored in: /Users/jiangyy/Codes/AI/LangChainCn/03-Samples/chroma/sql




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{"tc": "房间宽表", "te": "data_wide_s_Room", "cc": "补充协议总价", "ce": "BcxyTotal"}

{"tc": "房间宽表", "te": "data_wide_s_Room", "cc": "项目名称", "ce": "ParentProjName"}

{"tc": "房间宽表", "te": "data_wide_s_Room", "cc": "实际交房日期", "ce": "SjjfDate"}

{"tc": "房间宽表", "te": "data_wide_s_Room", "cc": "标准建筑单价", "ce": "BldPrice"}

Question: 这段json中tc表示数据表中文名, te标书数据表名, cc表示字段中文名, ce表示字段名。
交易宽表的表名是什么？ 
Helpful Answer:

> Finished chain.

> Finished chain.


' data_wide_s_Room'